In [30]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

In [2]:
os.getcwd()

'/Users/mahtablashgari/Desktop/GemeinsamRepository/Skin_cancer_diagnosis_tool/Notebooks'

In [9]:
ls


Dataset/            Notebooks/          models/             test/
Dockerfile          README.md           requirements.txt
LICENSE             docker-compose.yml  src/


In [12]:
cd dataverse_files

/Users/mahtablashgari/Desktop/GemeinsamRepository/Skin_cancer_diagnosis_tool/Dataset/dataverse_files


/Users/mahtablashgari/Library/Python/3.12/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
import os
import pandas as pd

data = pd.read_csv('HAM10000_metadata.csv')


In [17]:

imputer = SimpleImputer(strategy='mean')
data['age'] = imputer.fit_transform(data[['age']])

label_encoder_sex = LabelEncoder()
data['sex'] = label_encoder_sex.fit_transform(data['sex'])

label_encoder_localization = LabelEncoder()
data['localization'] = label_encoder_localization.fit_transform(data['localization'])

dx_classes = {'nv': 0 , 'mel': 6, 'bkl': 3, 'bcc': 4, 'vasc': 2, 'akiec':5, 'df':1}
data['Class']= data['dx'].map(dx_classes)

X = data[['age', 'sex', 'localization']]
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_GB = GradientBoostingClassifier(random_state=42)
model_GB.fit(X_train, y_train)

y_pred = model_GB.predict(X_test)
y_pred_proba = model_GB.predict_proba(X_test)


print("Genauigkeit:", accuracy_score(y_test, y_pred))
print("Klassifikationsbericht:")
print(classification_report(y_test, y_pred))

# ROC-AUC 
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6])
roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class="ovr")
print(f"ROC-AUC-Score: {roc_auc}")

vergleich = pd.DataFrame({
    'Tatsächlicher Wert': y_test,
    'Vorhergesagter Wert': y_pred
})

for i in range(len(y_test_binarized[0])):
    vergleich[f'Wahrscheinlichkeit Klasse {i}'] = y_pred_proba[:, i]

print(vergleich.head(2))

Genauigkeit: 0.7039440838741887
Klassifikationsbericht:
              precision    recall  f1-score   support

           0       0.75      0.97      0.85      1338
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        21
           3       0.42      0.36      0.38       228
           4       0.40      0.02      0.04        93
           5       0.33      0.09      0.14        69
           6       0.37      0.12      0.18       226

    accuracy                           0.70      2003
   macro avg       0.33      0.22      0.23      2003
weighted avg       0.62      0.70      0.64      2003

ROC-AUC-Score: 0.8333333722168428
      Tatsächlicher Wert  Vorhergesagter Wert  Wahrscheinlichkeit Klasse 0  \
1617                   6                    3                     0.067020   
8128                   0                    0                     0.867997   

      Wahrscheinlichkeit Klasse 1  Wahrscheinlichkeit Klasse 2  \
1617        

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [18]:
joblib.dump(model_GB, 'model_GB.pkl')
print("Modell erfolgreich gespeichert!")

Modell erfolgreich gespeichert!


In [21]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Datenvorverarbeitung
imputer = SimpleImputer(strategy='mean')
data['age'] = imputer.fit_transform(data[['age']])

label_encoder_sex = LabelEncoder()
data['sex'] = label_encoder_sex.fit_transform(data['sex'])

label_encoder_localization = LabelEncoder()
data['localization'] = label_encoder_localization.fit_transform(data['localization'])

dx_classes = {'nv': 0 , 'mel': 6, 'bkl': 3, 'bcc': 4, 'vasc': 2, 'akiec': 5, 'df': 1}
data['Class'] = data['dx'].map(dx_classes)

X = data[['age', 'sex', 'localization']]
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Modell 1: RF
model_RF = RandomForestClassifier(random_state=42)
model_RF.fit(X_train, y_train)
y_pred_RF = model_RF.predict(X_test)
y_pred_proba_RF = model_RF.predict_proba(X_test)

# Modell 2: Gradient Boosting
model_GB = GradientBoostingClassifier(random_state=42)
model_GB.fit(X_train, y_train)
y_pred_GB = model_GB.predict(X_test)
y_pred_proba_GB = model_GB.predict_proba(X_test)

# Modell 3: Support Vector Machine (SVM)
model_SVM = SVC(probability=True, random_state=42)
model_SVM.fit(X_train, y_train)
y_pred_SVM = model_SVM.predict(X_test)
y_pred_proba_SVM = model_SVM.predict_proba(X_test)

# Modell 4: K-Nearest Neighbors (KNN)
model_KNN = KNeighborsClassifier()
model_KNN.fit(X_train, y_train)
y_pred_KNN = model_KNN.predict(X_test)
y_pred_proba_KNN = model_KNN.predict_proba(X_test)

# Auswertung für alle Modelle
models = {
    "RF": (y_pred_RF, y_pred_proba_RF),
    "Gradient Boosting": (y_pred_GB, y_pred_proba_GB),
    "SVM": (y_pred_SVM, y_pred_proba_SVM),
    "KNN": (y_pred_KNN, y_pred_proba_KNN)
}

# Binarisierung der Zielvariable für ROC-AUC
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6])

for model_name, (y_pred, y_pred_proba) in models.items():
    print(f"\nModell: {model_name}")
    print("Genauigkeit:", accuracy_score(y_test, y_pred))
    print("Klassifikationsbericht:")
    print(classification_report(y_test, y_pred))
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class="ovr")
    print(f"ROC-AUC-Score: {roc_auc}")



Modell: RF
Genauigkeit: 0.7059410883674488
Klassifikationsbericht:
              precision    recall  f1-score   support

           0       0.76      0.97      0.85      1338
           1       0.00      0.00      0.00        28
           2       1.00      0.05      0.09        21
           3       0.44      0.33      0.38       228
           4       0.30      0.09      0.13        93
           5       0.25      0.09      0.13        69
           6       0.34      0.10      0.16       226

    accuracy                           0.71      2003
   macro avg       0.44      0.23      0.25      2003
weighted avg       0.63      0.71      0.64      2003

ROC-AUC-Score: 0.8288085038901901

Modell: Gradient Boosting
Genauigkeit: 0.7039440838741887
Klassifikationsbericht:
              precision    recall  f1-score   support

           0       0.75      0.97      0.85      1338
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        21
 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [22]:
import joblib

# Speichern der Modelle
joblib.dump(model_RF, 'model_Random_Forest.pkl')
joblib.dump(model_GB, 'model_gradient_boosting.pkl')
joblib.dump(model_SVM, 'model_svm.pkl')
joblib.dump(model_KNN, 'model_knn.pkl')


['model_knn.pkl']

In [33]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score

y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6])

models = {
    "RF": model_RF,
    "Gradient Boosting": model_GB,
    "SVM": model_SVM,
    "KNN": model_KNN
}

# Initialize an empty list to store the results
results = []

# Loop through each model for evaluation
for model_name, model in models.items():
     # Predictions for the test set
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    # Calculate metrics for evaluation
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class="ovr")

    # Store the results in the list
    results.append({
        "Modell": model_name,
        "Genauigkeit": accuracy,
        "F1-Score": f1,
        "ROC-AUC-Score": roc_auc
    })
    
results_df = pd.DataFrame(results)
print(results_df.head())


              Modell  Genauigkeit  F1-Score  ROC-AUC-Score
0                 RF     0.705941  0.641709       0.828809
1  Gradient Boosting     0.703944  0.636592       0.833333
2                SVM     0.667998  0.535038       0.652275
3                KNN     0.677983  0.629808       0.688642
